In [1]:
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler

from modis_tools.auth import add_earthdata_netrc, remove_earthdata_netrc

from datetime import datetime, timedelta
from pathlib import Path

### Use modis-tools to search and download MODIS granules

In [2]:
# Create an entry for Earthdata in the ~/.netrc file, only needs to be run once

#username = ""
#password = ""
#add_earthdata_netrc(username, password)

In [2]:
# Create a session
session = ModisSession()

In [19]:
# Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)

# Choose a collection to query
collections = collection_client.query(short_name="MYD02HKM", version="6.1") # Aqua 500m calibrated radiances
#collections = collection_client.query(short_name="MYD03", version="6.1") # Aqua geolocation fields
#collections = collection_client.query(short_name="MYD29", version="61") # Aqua sea ice

collections

[Collection(id='C1379758778-LAADS', title='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', dataset_id='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', coordinate_system='CARTESIAN', time_start='2002-07-04T00:45:00.000Z', updated=None, links=[CollectionLink(hreflang='en-US', href=AnyUrl('https://doi.org/10.5067/MODIS/MYD02HKM.061', scheme='https', host='doi.org', tld='org', host_type='domain', path='/10.5067/MODIS/MYD02HKM.061'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/mcst.gsfc/files/file_attachments/MODIS_L1B_ATBD_ver4.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/mcst.gsfc/files

In [20]:
# Query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

In [21]:
# Filter the selected granules via spatial and temporal parameters
# e.g.
# nigeria_bbox = [2.1448863675, 4.002583177, 15.289420717, 14.275061098]
# nigeria_granules = granule_client.query(start_date="2016-01-01", end_date="2018-12-31", bounding_box=nigeria_bbox)

# See https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html for more options

# Date range:
start_date = "2022-01-01"
end_date = "2022-12-31"

# Spatial extent:
labsea = [-65.0, 45.0, -40.0, 70.0] # bounding_box for Labrador Sea
points = ["-55.0,53.75", "-64.25,61.5", "-53.25,64.25", "-44.0,59.25"] # four approximate 'corners' of the Labrador Sea
points2 = ["-56.0,52.5", "-68.0,60.0", "-53.0,66.0", "-41.0,60.0"] # four corners of a larger area
points3 = ["-64.0,55.0", "-44.0,63.0"] # two ends of a line bisecting the Labrador Sea

# NB point co-ordinate is in lon, lat (opposite to earthdata web search!)
# Default option for point search is AND, i.e. all points will be within each granule

# Get list of granules fitting given criteria:
labsea2022_granules = granule_client.query(start_date=start_date, end_date=end_date, 
                                           day_night_flag="day", bounding_box=labsea) 
                                           #point=points2) 
                                            

# kwargs must be passed as dict if any options are to be used, e.g OR instead of AND
# e.g. 
# labsea2022_granules = granule_client.query(start_date=start_date, end_date=end_date, 
#                                           **{"point":["-52.5,57.5", "-62.4,57.5"], "options[point][or]":"true",
#                                              "day_night_flag":"day"})



In [22]:
# Check how many granules before downloading!
labsea2022_granules = list(labsea2022_granules) # make it a list as generator can only be used once
len(labsea2022_granules)

0

In [19]:
labsea2022_granules[0:10]

[]

In [8]:
# Specify download folder

folder = "AquaHkmLabSea2022b"
#folder = "AquaHkmLabSea2022b/geoloc"
filepath = Path("/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/"+folder)
filepath.mkdir(exist_ok=True, parents=True)

In [9]:
# Save the list of granules to a text file
with open(filepath / "granule_list.txt", "w") as f:
    for granule in labsea2022_granules:
        f.write(f"{granule}\n")

In [10]:
# Download the granules

GranuleHandler.download_from_granules(labsea2022_granules, session, path=str(filepath), threads=10)

Downloading:   0%|          | 0/71 [00:00<?, ?file/s]

[PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022352.1640.061.2022353162327.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022343.1650.061.2022344151151.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022331.1630.061.2022332152535.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022322.1640.061.2022323233701.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022319.1610.061.2022320182159.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022316.1540.061.2022317155640.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022310.1620.061.2022311152222.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/AquaHkmLabSea2022b/geoloc/MYD03.A2022307.1550.061.2022308152844.hdf'),
